# Препроцессинг и пайплайн

Для начала подключим библиотеки, которые нам понадобятся.

In [1]:
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.core.pylabtools import figsize

In [2]:
data = pd.read_csv('winequalityN.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('type', axis='columns'), data['type'], stratify=data['type'])

In [4]:
def dropper(data):
    data = data.drop(['alcohol', 'quality'], axis='columns')
    return data

In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()

def other_transformations(data):
    if "type" in data.keys():
        data["type"] = labelencoder.fit_transform(data["type"])
    
    for col in data.columns:
        mean = statistics.mean(data[col].dropna())
        data[col] = data[col].apply(lambda value: mean if pd.isnull(value) else value)
    return data

In [6]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.preprocessing import FunctionTransformer

pipeline = make_pipeline(
    FunctionTransformer(dropper, validate=False),                 # добавили удаление признаков в пайплайн;
                                                                  # параметр validate=False - чтобы функция не пыталась привести все признаки к числам
    FunctionTransformer(other_transformations, validate=False),   # добавили остальные трансформации признаков
    MinMaxScaler(),                               # нормализацию
    SVC(),          # и сам классификатор с наилучшими параметрами
)

pipeline

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function dropper at 0x0000022AE7C538B0>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function other_transformations at 0x0000022AECCD50D0>)),
                ('minmaxscaler', MinMaxScaler()), ('svc', SVC())])

In [7]:
pipeline.fit(X_train,  y_train)
pipeline.score(X_test, y_test)

0.9944615384615385

Посмотрим на распределения и возможные зависимости признаков.

## Сохранение модели
Последнее, что осталось сделать, - сохранить обученную модель. Тут всё очень просто: нам потребуется библиотека pickle и сама обученная модель.

In [8]:
import pickle
# нам нужно открыть целевой файл для записи (параметр w) в двоичном виде (параметр b), чтобы записать туда пикл модели
with open("lab2_model.pickle", 'wb') as opened_file:
    pickle.dump(pipeline, opened_file)

А теперь сымитируем получение модели другим человеком - перезапустим ядро ноутбука (все наши текущие переменные сотрутся из памяти) и загрузим готовую модель

In [9]:
import pandas as pd
import pickle

with open("lab2_model.pickle", 'rb') as f:
    model = pickle.load(f)
    
model

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function dropper at 0x0000022AE7C538B0>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function other_transformations at 0x0000022AECCD50D0>)),
                ('minmaxscaler', MinMaxScaler()), ('svc', SVC())])